<div style="width: 100%; clear: both;">
<div style="float: left; width: 50%;">
<img src="http://www.uoc.edu/portal/_resources/common/imatges/marca_UOC/UOC_Masterbrand.jpg", align="left">
</div>
</div>
<div style="float: right; width: 50%;">
<p style="margin: 0; padding-top: 22px; text-align:right;">M2.877 Anàlisi de sentiments i textos</p>
<p style="margin: 0; text-align:right;">Màster Universitari en Ciència de Dades (Data science)</p>
<p style="margin: 0; text-align:right; padding-button: 100px;">Estudis d'Informàtica, Multimèdia i Telecomunicacions</p>
</div>
</div>
<div style="width: 100%; clear: both;">
<div style="width:100%;">&nbsp;</div>


# PAC 1: Processament i anàlisi de la informació textual

En aquesta pràctica revisarem i aplicarem els coneixements apresos als mòduls de l'1 al 2. Concretament tractarem 3 temes.

<ul>
<li>1. Obtenció de dades a partir de la informació textual
<li>2. Detecció de temes
<li>3. Classificació de textos
</ul>

El propòsit de la pràctica és descobrir trets característics de les opinions sobre restaurants amb les eines explicades dels mòduls 1 al 2. A més, veurem si és possible classificar automàticament una opinió com a positiva o negativa amb mètodes de machine learning. Farem servir el dataset <i>restaurants_reviews.csv</i>, extret d'una plataforma d'expressió d'opinions. Aquest dataset conté opinions sobre restaurants en anglès. El dataset s'organitza en 10 columnes:

<b>business_id</b>: identificador del restaurant<br>
<b>date</b>: data de publicació de l'opinió<br>
<b>review_id</b>: identificador de l'opinió<br>
<b>stars</b>: qualificació o valoració del restaurant en estrelles<br>
<b>text</b>: text de l'opinió<br>
<b>type</b>: tipus de text<br>
<b>user_id</b>: identificador d'usuari<br>
<b>cool, useful </b> i <b>funny</b>: Nombre de valoracions que han realitzat els usuaris de la plataforma per a aquests tres criteris.

In [1]:
import warnings
warnings.filterwarnings('ignore')

# Preparació del dataset

In [2]:
from google.colab import drive
drive.mount('/content/drive')
%cd "/content/drive/MyDrive/ColabNotebooks" 

Mounted at /content/drive
/content/drive/MyDrive/ColabNotebooks


In [3]:
import pandas as pd
# Obrir el fitxer de comentaris:
try:
  df = pd.read_csv('/content/drive/MyDrive/ColabNotebooks/restaurants_reviews.csv')
except:
  df = pd.read_csv('/content/restaurants_reviews.csv')
df.head()

,business_id,date,review_id,stars,text,type,user_id,cool,useful,funny
0,9yKzy9PApeiPPOUJEtnvkg,2011-01-26,fWKvX83p0-ka4JS3dc6E5A,5,My wife took me here on my birthday for breakf...,review,rLtl8ZkDX5vH5nAx9C3q5Q,2,5,0
1,ZRJwVLyzEJq1VAihDhYiow,2011-07-27,IjZ33sJrzXqU-0X6U8NwyA,5,I have no idea why some people give bad review...,review,0a2KyEL0d3Yb1V6aivbIuQ,0,0,0
2,6oRAC4uyJCsJl1X0WZpVSA,2012-06-14,IESLBzqUCLdSzSqm0eCSxQ,4,love the gyro plate. Rice is so good and I als...,review,0hT2KtfLiobPvh6cDC8JQg,0,1,0
3,-yxfBYGB6SEqszmxJxd97A,2007-12-13,m2CKSsepBCoRYWxiRUsxAg,4,"Quiessence is, simply put, beautiful. Full wi...",review,sqYN3lNgvPbPCTRsMFu27g,4,3,1
4,zp713qNhx8d9KCJJnrw1xA,2010-02-12,riFQ3vxNpP4rWLk_CSri2A,5,Drop what you're doing and drive here. After I...,review,wFweIWhv2fREZV_dYkz_1g,7,7,4


Per fer la pràctica, només necessitarem els textos i les valoracions en estrelles. Per tant, eliminem les columnes innecessàries i ens quedarem només amb les columnes <b>stars</b> i <b>text</b>.

In [4]:
#############################################
df_star_texts=pd.DataFrame()
df_star_texts=df.drop(['business_id','date','review_id','type','user_id','cool','useful','funny'], axis=1)
df_star_texts.head(20)
#############################################

,stars,text
0,5,My wife took me here on my birthday for breakf...
1,5,I have no idea why some people give bad review...
2,4,love the gyro plate. Rice is so good and I als...
3,4,"Quiessence is, simply put, beautiful. Full wi..."
4,5,Drop what you're doing and drive here. After I...
5,4,Definitely come for Happy hour! Prices are ama...
6,5,Nobuo shows his unique talents with everything...
7,5,Wonderful Vietnamese sandwich shoppe. Their ba...
8,5,They have a limited time thing going on right ...
9,4,I'm 2 weeks new to Phoenix. I looked up Irish ...


Treballarem amb les opinions que tenen les qualificacions més altes i les més baixes. Les opinions amb la qualificació més gran que 3 seran etiquetades amb '1', mentre que etiquetarem amb '0' les opinions que tinguin una qualificació menor que 3. Les etiquetes s'assignen a la columna <b>sentiment</b>.

In [5]:
#############################################
import numpy as np

df_star_texts = df_star_texts[df_star_texts['stars'] != 3]
df_star_texts['sentiment'] = np.where(df_star_texts['stars'] > 3, '1', '0')

unique_count = df_star_texts['sentiment'].value_counts()
print(unique_count)
df_star_texts.head(11)
#############################################

1    4694
0    1157
Name: sentiment, dtype: int64


,stars,text,sentiment
0,5,My wife took me here on my birthday for breakf...,1
1,5,I have no idea why some people give bad review...,1
2,4,love the gyro plate. Rice is so good and I als...,1
3,4,"Quiessence is, simply put, beautiful. Full wi...",1
4,5,Drop what you're doing and drive here. After I...,1
5,4,Definitely come for Happy hour! Prices are ama...,1
6,5,Nobuo shows his unique talents with everything...,1
7,5,Wonderful Vietnamese sandwich shoppe. Their ba...,1
8,5,They have a limited time thing going on right ...,1
9,4,I'm 2 weeks new to Phoenix. I looked up Irish ...,1


# Preprocessament

Abans de treballar amb els textos de les opinions, cal netejar-los de caràcters com els salts de línia (e.g: *Should be called house of deliciousness!\r\n\r*)

In [6]:
#############################################
#  df_star_texts.rstrip('\n')   #Només el·limina les subcadenes de la posició inicial i final de la cadena.
import re

df_star_texts['text'] = df_star_texts['text'].apply(lambda x: re.sub('[\r\n]+', ' ', x))

df_star_texts['text'].loc[0]
#############################################

'My wife took me here on my birthday for breakfast and it was excellent.  The weather was perfect which made sitting outside overlooking their grounds an absolute pleasure.  Our waitress was excellent and our food arrived quickly on the semi-busy Saturday morning.  It looked like the place fills up pretty quickly so the earlier you get here the better. Do yourself a favor and get their Bloody Mary.  It was phenomenal and simply the best I\'ve ever had.  I\'m pretty sure they only use ingredients from their garden and blend them fresh when you order it.  It was amazing. While EVERYTHING on the menu looks excellent, I had the white truffle scrambled eggs vegetable skillet and it was tasty and delicious.  It came with 2 pieces of their griddled bread with was amazing and it absolutely made the meal complete.  It was the best "toast" I\'ve ever had. Anyway, I can\'t wait to go back!'

També cal eliminar els dobles espais:

In [7]:
#############################################
df_star_texts['text'] = df_star_texts['text'].apply(lambda x: re.sub(' +', ' ', x))
df_star_texts['text'].loc[0]
#############################################


'My wife took me here on my birthday for breakfast and it was excellent. The weather was perfect which made sitting outside overlooking their grounds an absolute pleasure. Our waitress was excellent and our food arrived quickly on the semi-busy Saturday morning. It looked like the place fills up pretty quickly so the earlier you get here the better. Do yourself a favor and get their Bloody Mary. It was phenomenal and simply the best I\'ve ever had. I\'m pretty sure they only use ingredients from their garden and blend them fresh when you order it. It was amazing. While EVERYTHING on the menu looks excellent, I had the white truffle scrambled eggs vegetable skillet and it was tasty and delicious. It came with 2 pieces of their griddled bread with was amazing and it absolutely made the meal complete. It was the best "toast" I\'ve ever had. Anyway, I can\'t wait to go back!'

# 1. Obtenció de dades a partir d'informació textual (4 punts)

## 1.1 Trobar col·locacions (2 punts)

Recordem que les col·locacions són termes multiparaula, és a dir, seqüències de paraules que, en conjunt, tenen un significat que difereix significativament del significat de cada paraula individual (e.g. New York té un significat diferent del que es pot derivar de New i de York).

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Exercici:</strong>  Calculeu els millors bigrames i trigrames de les opinions. (1 punt)
</div>

In [8]:
#############################################
# carreguem les llibreries
import nltk
from nltk import pos_tag
from nltk.tokenize import word_tokenize
from nltk.collocations import *
import re
nltk.download('punkt')   #punkt es una eina de biblioteca nltk per tokenitzar documents de texte
nltk.download("stopwords")
nltk.download('averaged_perceptron_tagger')    # Modul del Perceptron Tagger
#############################################

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [9]:
#############################################
#Import de la llista de stopwords en anglés de la libreria NLTK.
stopwords = nltk.corpus.stopwords.words('english')
#Afegim stopwords
stopwords = stopwords + ['unknown', 've', 'hadn', 'll', 'didn', 'isn', 'doesn', 'hasn' ]
#############################################

A partir de l'ordre help(nltk.collocations.BigramAssocMeasures) explora la classe BigramAssocMeasures del mòdul nltk.metrics.association i revisa les definicions de les mètriques de Likelihood Ratio (likelihood_ratio) i de Pointwise Mutual Information (pmi) del llibre Foundations of Statistical Natural Language Processing (Manning & Schutze).

<i>Primer pas</i>: Obtenir els tokens del text de les opinions. Etiqueta aquests tokens pel seu PoS. (0.5 punts)

In [10]:
#############################################
help(nltk.collocations.BigramAssocMeasures)

#############################################

Help on class BigramAssocMeasures in module nltk.metrics.association:

class BigramAssocMeasures(NgramAssocMeasures)
 |  A collection of bigram association measures. Each association measure
 |  is provided as a function with three arguments::
 |  
 |      bigram_score_fn(n_ii, (n_ix, n_xi), n_xx)
 |  
 |  The arguments constitute the marginals of a contingency table, counting
 |  the occurrences of particular events in a corpus. The letter i in the
 |  suffix refers to the appearance of the word in question, while x indicates
 |  the appearance of any word. Thus, for example:
 |  
 |  - n_ii counts ``(w1, w2)``, i.e. the bigram being scored
 |  - n_ix counts ``(w1, *)``
 |  - n_xi counts ``(*, w2)``
 |  - n_xx counts ``(*, *)``, i.e. any bigram
 |  
 |  This may be shown with respect to a contingency table::
 |  
 |              w1    ~w1
 |           ------ ------
 |       w2 | n_ii | n_oi | = n_xi
 |           ------ ------
 |      ~w2 | n_io | n_oo |
 |           ------ ------
 |  

In [11]:
#############################################
print(df_star_texts.text)

print('Hi ha nulls a la columna text?',df_star_texts['text'].isnull().values.any())
# Treiem els texts buits
#df_Text = [df_t for df_t in df_star_texts['text'].to_list() if df_t != 'empty']
#print(type(df_Text))

#Concatenem els valors del camp 'text' per obtenir un text pla.
df_Text = " ".join(df_star_texts.text.tolist()) #text pla

# Obtenir els tokens del text de les opinions
tokens_text = [w for w in word_tokenize(df_Text.lower()) if re.match('^[a-z]*$', w)]   # No li aplicaria el filtre de stop words
print(tokens_text[:100])

# Etiquetar els tokens pel seu PoS
tokens_tagged = pos_tag(tokens_text)
print(tokens_tagged[:20])

#############################################



0       My wife took me here on my birthday for breakf...
1       I have no idea why some people give bad review...
2       love the gyro plate. Rice is so good and I als...
3       Quiessence is, simply put, beautiful. Full win...
4       Drop what you're doing and drive here. After I...
                              ...                        
6905    Judging by some of the reviews, maybe I went o...
6907    Should be called house of deliciousness! I cou...
6908    I recently visited Olive and Ivy for business ...
6909    My nephew just moved to Scottsdale recently so...
6910    4-5 locations.. all 4.5 star average.. I think...
Name: text, Length: 5851, dtype: object
Hi ha nulls a la columna text? False
['my', 'wife', 'took', 'me', 'here', 'on', 'my', 'birthday', 'for', 'breakfast', 'and', 'it', 'was', 'excellent', 'the', 'weather', 'was', 'perfect', 'which', 'made', 'sitting', 'outside', 'overlooking', 'their', 'grounds', 'an', 'absolute', 'pleasure', 'our', 'waitress', 'was', 'exce

<i>Segon pas</i>: Calcular els 1000 millors bigrames i els 1000 millors trigrames a partir dels tokens etiquetats (e.g. [(Basic, JJ), ...]) del text. Utilitza la mètrica PMI o Likehood Ratio. Has d'indicar per què has triat una i no una altra. (0.5 punts)

<b>Atenció</b>: Dels 1000 bigrames i trigrames, tria els que no comencen ni acaben amb una stopword.

Recordem la classificació d'etiquetes PoS.

<b>Etiquetas PoS</b>

<ul>
<li>DT: Determinant</li>
<li>JJ: Adjectiu</li>
<li>NN: Nom en singular</li>
<li>NNS: Nom en plural</li>
<li>VBD: Verb en passat</li>
<li>VBG: Verb en gerundi</li>
<li>MD: Verb modal</li>
<li>IN: Preposició o conjunció subordinada</li>
<li>PRP: Pronom</li>
<li>RB: Adverbi</li>
<li>RP: Partícula</li>    
<li>CC: Conjunció coordinada</li>
<li>CD: Numeral</li>
</ul>

In [12]:
from os import O_NONBLOCK
#############################################
# Es carreguen les mides pel càlcul de bigrams i trigrams
bigram_measures = nltk.collocations.BigramAssocMeasures()
trigram_measures = nltk.collocations.TrigramAssocMeasures()

def is_good_ngram(candidate):
    test = True
    if candidate[0][0] in stopwords or candidate[-1][0] in stopwords:
        test = False
    return test

def convert_tupla_string(ngram_tuples):
    val = []
    for nt in ngram_tuples:
        val.append(nt[0])
    return " ".join(val)

"""
El PMI (Pointwise Mutual Information) mesura la relació entre dues paraules en un corpus de text, comparant la probabilitat 
d'aparició conjunta de les paraules amb la seva probabilitat d'aparició individual. El PMI és una bona opció si es vol mesurar 
la força de la relació entre paraules específiques en les ressenyes del restaurant, per exemple, si es vol saber si els 
clients que mencionen "menjar" també mencionen "servei" amb freqüència.

D'altra banda, el Likelihood Ratio mesura la freqüència de la paraula en el corpus de text i la freqüència d'aquesta paraula 
en un subconjunt específic del corpus (com ara les ressenyes positives o negatives). El Likelihood Ratio és una opció adequada 
si es vol mesurar la probabilitat que una paraula aparegui en una ressenya positiva en comparació amb la probabilitat que 
aparegui en una ressenya negativa.

En resum, si es vol mesurar la relació entre paraules específiques en les ressenyes del restaurant, seria més adequat utilitzar 
el PMI. Si es vol mesurar la probabilitat que una paraula aparegui en una ressenya positiva en comparació amb una negativa, 
seria més adequat utilitzar el Likelihood Ratio. Com que el nostre objectiu és descobrir trets característics de les opinions 
sobre restaurants i classificar si una opinió és positiva o negativa, ens decantem pel Likelihood Ratio.
"""

################ NGRAMS AMB LIKEHOOD RATIO ##############################################33
text_bigramcandidates_lr = BigramCollocationFinder.from_words(tokens_tagged).nbest(bigram_measures.likelihood_ratio,1000)
print('bigrams found',len(text_bigramcandidates_lr))
print('text_bigramcandidates_lr: ',text_bigramcandidates_lr[:10])
text_trigramcandidates_lr = TrigramCollocationFinder.from_words(tokens_tagged).nbest(trigram_measures.likelihood_ratio,1000)
print('trigrams found',len(text_trigramcandidates_lr))
print('text_trigramcandidates_lr: ',text_trigramcandidates_lr[:10])

# Filtratge dels ngrams que comencin o acabin amb un stopword
text_bigram_lr_filter = [t_bg for t_bg in text_bigramcandidates_lr if is_good_ngram(t_bg) == True]
print('text_bigram_lr_filter: ',text_bigram_lr_filter[:10])
text_trigram_lr_filter = [t_tg for t_tg in text_trigramcandidates_lr if is_good_ngram(t_tg) == True]
print('text_trigram_lr_filter: ',text_trigram_lr_filter[:10])

# Convertim tuplas a strings
text_bigram_lr_string = [convert_tupla_string(t_bgrn) for t_bgrn in text_bigram_lr_filter]
print('text_bigram_lr_string: ',text_bigram_lr_string[:10])
text_trigram_lr_string = [convert_tupla_string(t_tgrn) for t_tgrn in text_trigram_lr_filter]
print('text_trigram_lr_string: ',text_trigram_lr_string[:10])

bigrams found 1000
text_bigramcandidates_lr:  [(('this', 'DT'), ('place', 'NN')), (('it', 'PRP'), ('was', 'VBD')), (('if', 'IN'), ('you', 'PRP')), (('happy', 'JJ'), ('hour', 'NN')), (('i', 'NN'), ('had', 'VBD')), (('the', 'DT'), ('food', 'NN')), (('the', 'DT'), ('best', 'JJS')), (('we', 'PRP'), ('were', 'VBD')), (('the', 'DT'), ('the', 'DT')), (('a', 'DT'), ('little', 'JJ'))]
trigrams found 1000
text_trigramcandidates_lr:  [(('this', 'DT'), ('place', 'NN'), ('is', 'VBZ')), (('a', 'DT'), ('this', 'DT'), ('place', 'NN')), (('this', 'DT'), ('place', 'NN'), ('to', 'TO')), (('the', 'DT'), ('it', 'PRP'), ('was', 'VBD')), (('the', 'DT'), ('this', 'DT'), ('place', 'NN')), (('about', 'IN'), ('this', 'DT'), ('place', 'NN')), (('love', 'VBP'), ('this', 'DT'), ('place', 'NN')), (('this', 'DT'), ('place', 'NN'), ('has', 'VBZ')), (('this', 'DT'), ('place', 'NN'), ('for', 'IN')), (('this', 'DT'), ('place', 'NN'), ('a', 'DT'))]
text_bigram_lr_filter:  [(('happy', 'JJ'), ('hour', 'NN')), (('first', 'JJ

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Exercici:</strong> Detectar ngrames que compleixen el patró sintàctic d'un sintagma nominal (e.g: adjectiu + nom en singular/plural i nom + nom) (0.5 punts)
</div>

In [13]:
#############################################
def obte_string_form(list_tuple):
    words = [cti[0] for cti in list_tuple]
    string_form = "_".join(words)
    return string_form

"""
NP: {<JJ><NN>|<JJ><NNS>}: Un sintagma nominal (NP) pot ser format per un adjectiu (JJ) seguit d'un nom singular (NN) o un adjectiu seguit d'un nom plural (NNS).
NP: {<DT>?<JJ>*<NN>} : Un sintagma nominal pot ser format per un determinat (DT) opcional, zero o més adjectius (JJ) i un nom singular (NN).
NP: {<CD>?<JJ>*<NNS>} : Un sintagma nominal pot ser format per un nombre (CD) opcional, zero o més adjectius (JJ) i un nom plural (NNS).
NP: {<PRP>|<PRP$>|<POS>} : Un sintagma nominal pot ser format per un pronom personal (PRP), un pronom possessiu (PRP$) o un apòstrof posessiu (POS).
NP: {<NNP>+} : Un sintagma nominal pot ser format per un o més noms propis (NNP).
CN: {<NN><NN>}: Un sintagma nominal (CN) pot ser format per dos noms seguits (NN). (Nom compost o Compund name)
"""

def obte_np_pattern(tokens_tag):
    n_patterns = """
      NP: {<DT>?<JJ><NN>|<CD>?<JJ><NNS>|<PRP>|<PRP$>|<POS>|<NNP>+}   # Nominal Phrase
      CN: {<NN><NN>|<NN><NNS>}   # Compound name
      """
    n_parser = nltk.RegexpParser(n_patterns)
    
    # Create a list of trees, one per collocation
    trees = [n_parser.parse(collocation) for collocation in tokens_tag]
    
    # Extract the matches from the trees
    n_and_cn = []
    for i in range(len(tokens_tag)):
        collocation_matches = n_parser.parse(tokens_tag[i]).subtrees(filter=lambda x: x.label() in ['NP', 'CN'])
        for match in collocation_matches:
            n_and_cn.append(tuple(leaf[0] for leaf in match.leaves()))
        
    return n_and_cn

text_bigrams_and_trigrams = text_bigram_lr_filter + text_trigram_lr_filter

text_np_pattern = obte_np_pattern(text_bigrams_and_trigrams)
print(list(set(text_np_pattern[:10])))
#############################################

[('first', 'time'), ('sweet', 'potato'), ('next', 'time'), ('french', 'toast'), ('strip', 'mall'), ('wait', 'staff'), ('mexican', 'food'), ('last', 'night'), ('ice', 'cream'), ('happy', 'hour')]


<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Exercici:</strong> Detectar col·locacions amb un model de detecció de frases, amb el mòdul Phraser de Gensim. Entrena el model amb totes les opinions (0,5 punts)
</div>

<i>Primer pas</i>: Convertir les opinions en una llista de phrases. Les phrases no han de ser stopwords. Tampoc no han de començar ni acabar amb una stopword. (0.5 punts)

In [14]:
#############################################
from gensim.models.phrases import Phrases, Phraser
from gensim.utils import simple_preprocess
#############################################

In [60]:
simple_preprocess(df_star_texts['text'].loc[0])[6] in stopwords

True

In [61]:
#############################################
# Define a function to preprocess the text
def preprocess_text(text):
    # Tokenize the text
    tokens = simple_preprocess(text)
    # Remove the tokens that are stopwords or colocations that start or end with stopword
    good_tokens = []
    for token in tokens:
      if token not in stopwords:
        good_tokens.append(token)
      elif '_' in token:
        if token.split('_')[0] not in stopwords and token.split('_')[1] not in stopwords:
          good_tokens.append(token)
      # Return the remaining tokens
    return good_tokens

# Preprocess the text
print(df_star_texts['text'].loc[:10])
texts = [preprocess_text(text) for text in df_star_texts['text']]
print(texts[:10])

#############################################


0     My wife took me here on my birthday for breakf...
1     I have no idea why some people give bad review...
2     love the gyro plate. Rice is so good and I als...
3     Quiessence is, simply put, beautiful. Full win...
4     Drop what you're doing and drive here. After I...
5     Definitely come for Happy hour! Prices are ama...
6     Nobuo shows his unique talents with everything...
7     Wonderful Vietnamese sandwich shoppe. Their ba...
8     They have a limited time thing going on right ...
9     I'm 2 weeks new to Phoenix. I looked up Irish ...
10    Was it worth the 21$ for a salad and small piz...
Name: text, dtype: object
[['wife', 'took', 'birthday', 'breakfast', 'excellent', 'weather', 'perfect', 'made', 'sitting', 'outside', 'overlooking', 'grounds', 'absolute', 'pleasure', 'waitress', 'excellent', 'food', 'arrived', 'quickly', 'semi', 'busy', 'saturday', 'morning', 'looked', 'like', 'place', 'fills', 'pretty', 'quickly', 'earlier', 'get', 'better', 'favor', 'get', 'bloo

## 1.2 Vectoritzar paraules i termes (2 punts)

Explorarem la vectorització de paraules i termes amb el mètode Word2Vec.

Recordem que el paquet gensim implementa un mètode per entrenar models Word2Vec.

In [16]:
import gensim

In [17]:
#############################################
from gensim.models.phrases import Phraser
from gensim.models import Word2Vec
#############################################



<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Exercici:</strong> Obtenir targets de les opinions i els seus aspectes utilitzant el model word2vec (2 punts)
</div>

<i>Primer pas</i>: Convertir les phrases de cada oració en un token. Ho farem concatenant els tokens de la phrase amb el caràcter '_' (e.g: 'scrambled eggs' -> 'scrambled_eggs'). Aleshores, a cada oració substituïm els bigrames que són phrases per la forma tokenitzada (e.g: I made scrambled eggs -> I made scrambled_eggs). Així, les col·locacions formaran part del vocabulari del model word2vec que generarem.

In [62]:
#############################################
"""
Utilitzarem el model Phrases que detecta le sol·locacions habituals en el corpus i ens retorna les phrases
amb les col·locacions detectades
"""
# Train the phrase model
model = Phrases(texts, min_count=1, threshold=2, delimiter='_')

# Use the phrase model to transform the text accordingly
model_phraser = Phraser(model)
texts_phraser = [model_phraser[text] for text in texts]

print(texts_phraser[:100])
#############################################


[['wife_took', 'birthday', 'breakfast', 'excellent', 'weather_perfect', 'made', 'sitting_outside', 'overlooking', 'grounds', 'absolute', 'pleasure_waitress', 'excellent_food', 'arrived_quickly', 'semi', 'busy_saturday', 'morning', 'looked_like', 'place', 'fills', 'pretty_quickly', 'earlier_get', 'better', 'favor_get', 'bloody_mary', 'phenomenal', 'simply_best', 'ever', 'pretty_sure', 'use_ingredients', 'garden', 'blend', 'fresh', 'order', 'amazing_everything', 'menu_looks', 'excellent', 'white_truffle', 'scrambled_eggs', 'vegetable', 'skillet', 'tasty', 'delicious', 'came_pieces', 'griddled', 'bread_amazing', 'absolutely_made', 'meal_complete', 'best', 'toast_ever', 'anyway', 'wait_go', 'back'], ['idea_people', 'give', 'bad_reviews', 'place', 'goes_show', 'please_everyone', 'probably', 'griping', 'something', 'fault', 'many_people', 'like', 'case', 'friend_arrived', 'pm', 'past_sunday', 'pretty_crowded', 'thought', 'sunday_evening', 'thought_would', 'wait', 'forever_get', 'seat', 'said

<i>Segon pas</i>: crear una sentence stream on tots els tokens de les oracions estan lematitzats. Els tokens no poden ser stopwords ni tenir un stopword al començament o al final. Per simplificar la tasca, considerem que el lema d'una col·locació no canvia i el seu PoS és col. (e.g: ['The guests like scrambled eggs', 'The rooms were dirty'] -> [['the', 'guest', 'like', 'scrambled_eggs], ['the', 'room', 'be ', 'dirty']]). (1 punt)

In [63]:
#############################################
from nltk.stem.wordnet import WordNetLemmatizer

nltk.download('wordnet')

# Tornem a filtrar ja que el model Phrase pot haver creat col·locacions amb stopwords
def is_good_token(token):
    test = True
    if '_' in token:
      if token.split('_')[0] in stopwords or token.split('_')[1] in stopwords:
          test = False
    else:
      if token in stopwords:
          test = False
    return test

# Associa NN, NNS, IN a 'n', VBG, VB, VBN, VBZ a 'v'  i finaltment   JJ, RBR, RB a 'n'
def obte_pos(pos):
    if re.match(r'^N',pos):
        wn_pos = 'n'
    elif re.match(r'^V',pos):
        wn_pos = 'v'
    else:
        wn_pos = 'n'
    return wn_pos

def wlemmatize(t,postag):
    lemmat = ""
    lem = WordNetLemmatizer()
    if '_' not in t:
        lemmat = lem.lemmatize(t,obte_pos(postag[0][1]))
    else:
        lemmat = t
    
    return lemmat

postag = {}

lemmatized_sentences = []
for sentence in texts_phraser:
  #print("sentence: ", sentence)
  new_sentence = []
  for token in sentence:
    if is_good_token(token):
      postag[token] = nltk.pos_tag([token])
      lemmatized_token = wlemmatize(token,postag[token])
      new_sentence.append(lemmatized_token)
  lemmatized_sentences.append(new_sentence)
  #print("new_sentence: ", new_sentence)

print(lemmatized_sentences[0])
#############################################

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


['wife_took', 'birthday', 'breakfast', 'excellent', 'weather_perfect', 'make', 'sitting_outside', 'overlook', 'ground', 'absolute', 'pleasure_waitress', 'excellent_food', 'arrived_quickly', 'semi', 'busy_saturday', 'morning', 'looked_like', 'place', 'fill', 'pretty_quickly', 'earlier_get', 'better', 'favor_get', 'bloody_mary', 'phenomenal', 'simply_best', 'ever', 'pretty_sure', 'use_ingredients', 'garden', 'blend', 'fresh', 'order', 'amazing_everything', 'menu_looks', 'excellent', 'white_truffle', 'scrambled_eggs', 'vegetable', 'skillet', 'tasty', 'delicious', 'came_pieces', 'griddle', 'bread_amazing', 'absolutely_made', 'meal_complete', 'best', 'toast_ever', 'anyway', 'wait_go', 'back']


<i>Tercer pas</i>: Crear un model word2vec de les opinions lematitzades. El model s'ha de dir w2v_opinions (0.5 punts)

In [64]:
#############################################
w2v_opinions = gensim.models.Word2Vec(
        sentences=lemmatized_sentences,
        vector_size=150,  # Mida de les dimensions del vector
        window=10, # Context window 
        min_count= 3, # frequencia mínima
        workers= 1,
        seed=1      # Valor inici predefinit per conservar la coherencia
)
#############################################


<i>Quart pas</i>: A partir del vocabulari del model word2vec, selecciona possibles aspectes de l'opinió (e.g: food) i llista termes semànticament relacionats amb aquests aspectes segons aquest model. (0.5 punts)

Obtenir el vocabulari:

In [65]:
"""
Quim: No estic segur que calgui aquesta verificació si ja hem filtrat abans els tokens

# Etiquetes que no poden tenir a l'inici i final el token
no_pos_ini_fin = ['DT', 'IN', 'PRP', 'CC', 'CD','MD', 'VBG', 'VBD', 'RP', 'RB']

#Método para evaluar la idoneidad de una palabra o una combinación de palabras
def is_good_candid(t,postag):
    v = False
    if '_' in t:
        tl = t.split('_')
        if re.match("^[a-z]+.*", tl[0]) and re.match("^[a-z]+.*", tl[-1]) and \
          tl[0] not in stopwords and tl[1] not in stopwords:
            if postag[0][1] not in no_pos_ini_fin and postag[-1][1] not in no_pos_ini_fin:
              v = True
    else:
        if t not in stopwords and re.match("^[a-z]+.*", t):
            if postag[0][1] not in no_pos_ini_fin:
                v = True
    return v

#Per verificar que una phrase es un terme
def is_phrase_a_term(phrase):
    test = False
    if phrase not in postag:
        pos = nltk.pos_tag(word_tokenize(phrase))
    else:
        pos = postag[phrase]
    if is_good_candid(phrase,pos):
        test = True
    return test
"""

vocabulary = list(w2v_opinions.wv.index_to_key)
print('vocabulary: ', vocabulary)

vocabulary:  ['place', 'food', 'good', 'get', 'go', 'like', 'great', 'order', 'one', 'time', 'try', 'make', 'love', 'come', 'restaurant', 'say', 'also', 'eat', 'well', 'back', 'menu', 'really', 'even', 'know', 'service', 'pizza', 'think', 'would', 'delicious', 'take', 'meal', 'little', 'drink', 'best', 'salad', 'nice', 'sandwich', 'u', 'burger', 'dinner', 'look', 'want', 'friend', 'much', 'dish', 'way', 'table', 'give', 'bar', 'taste', 'chicken', 'amaze', 'fry', 'serve', 'lunch', 'sauce', 'always', 'right', 'see', 'people', 'thing', 'enjoy', 'still', 'price', 'ask', 'day', 'first', 'night', 'though', 'happy_hour', 'experience', 'side', 'better', 'since', 'lot', 'flavor', 'wait', 'ever', 'beer', 'tasty', 'server', 'cheese', 'use', 'start', 'awesome', 'work', 'location', 'star', 'could', 'plate', 'never', 'fresh', 'two', 'favorite', 'find', 'sure', 'review', 'bread', 'visit', 'need', 'bad', 'special', 'area', 'everything', 'appetizer', 'dessert', 'bit', 'phoenix', 'meat', 'really_good', 

In [67]:
#############################################
# Busquem termes semànticament relacionats a food
term = 'food'
similar = w2v_opinions.wv.most_similar(term, topn=50)
print(similar)
#############################################

#############################################
# Busquem termes semànticament relacionats a drink
term = 'drink'
similar = w2v_opinions.wv.most_similar(term, topn=50)
print(similar)
#############################################

#############################################
# Busquem termes semànticament relacionats a staff
term = 'staff'
similar = w2v_opinions.wv.most_similar(term, topn=50)
print(similar)
#############################################

[('people', 0.9997833967208862), ('experience', 0.9997758269309998), ('go', 0.9997710585594177), ('never', 0.9997670650482178), ('review', 0.9997638463973999), ('take', 0.9997622966766357), ('work', 0.9997533559799194), ('eat', 0.9997405409812927), ('location', 0.9997376203536987), ('show', 0.9997372627258301), ('friend', 0.9997351169586182), ('way', 0.9997338652610779), ('wait', 0.9997308254241943), ('service', 0.9997307062149048), ('since', 0.9997283816337585), ('staff', 0.9997283816337585), ('food_good', 0.9997278451919556), ('atmosphere', 0.9997236132621765), ('bad', 0.9997235536575317), ('great_place', 0.9997207522392273), ('first_time', 0.9997170567512512), ('customer', 0.9997164011001587), ('would', 0.9997154474258423), ('server', 0.9997150301933289), ('busy', 0.9997149109840393), ('sure', 0.9997145533561707), ('first', 0.999714195728302), ('change', 0.9997122883796692), ('drink', 0.9997117519378662), ('visit', 0.9997113347053528), ('open', 0.9997096657752991), ('crowd', 0.99970

In [68]:
#############################################
# Busquem termes semànticament relacionats a quality
term = 'quality'
similar = w2v_opinions.wv.most_similar(term, topn=50)
print(similar)
#############################################

#############################################
# Busquem termes semànticament relacionats a good
term = 'great'
similar = w2v_opinions.wv.most_similar(term, topn=50)
print(similar)
#############################################

#############################################
# Busquem termes semànticament relacionats a bad
term = 'bad'
similar = w2v_opinions.wv.most_similar(term, topn=50)
print(similar)
#############################################


[('style', 0.9999328255653381), ('bit', 0.9999289512634277), ('option', 0.9999268054962158), ('full', 0.9999220967292786), ('enjoy', 0.999921977519989), ('super', 0.9999212622642517), ('whole', 0.9999212622642517), ('wonderful', 0.9999203085899353), ('although', 0.9999197721481323), ('however', 0.9999193549156189), ('big', 0.9999191761016846), ('course', 0.9999186992645264), ('enough', 0.9999186396598816), ('two', 0.999918520450592), ('without', 0.9999178051948547), ('offer', 0.9999175071716309), ('definitely', 0.9999171495437622), ('type', 0.999916672706604), ('recommend', 0.9999163150787354), ('perfectly', 0.9999157786369324), ('pretty', 0.999915361404419), ('small', 0.9999153017997742), ('selection', 0.999914824962616), ('choice', 0.9999142289161682), ('either', 0.9999140501022339), ('anything', 0.9999138116836548), ('perfect', 0.9999138116836548), ('oh', 0.99991375207901), ('stuff', 0.9999136924743652), ('probably', 0.9999134540557861), ('ate', 0.9999130368232727), ('house', 0.9999

# 2. Detecció de temes (4 punts)

En aquests apartats explorarem quins són els tòpics tractats a les opinions.

## 2.1 Exploració dels temes amb WordNet (2 punts)

En aquest apartat accedirem a Wordnet a través de la llibreria nltk.

In [25]:
from nltk.corpus import wordnet as wn

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Exercici:</strong> Comprova si, segons Wordnet, hi ha aspectes que estan allunyats semànticament del sentit del target, encara que en el model word2vec siguin similars. Comproveu-ho calculant la similitud de Wu and Palmer entre el sentit de wordnet 'restaurant.n.01' i alguns dels seus aspectes. (1 punt)
</div>

In [71]:
#############################################
restaurant = wn.synset('restaurant.n.01')
food = wn.synset('food.n.01')
menu = wn.synset('menu.n.01')
waiter = wn.synset('waiter.n.01')
ambiance = wn.synset('ambiance.n.01')
reservation = wn.synset('reservation.n.01')

print("Hiperònims de restaurant: ", restaurant.hypernyms())
print("Hiperònims de food: ", food.hypernyms())
print("Hiperònims de menu: ", menu.hypernyms())
print("Hiperònims de waiter: ", waiter.hypernyms())
print("Hiperònims de ambiance: ", ambiance.hypernyms())
print("Hiperònims de reservation: ", reservation.hypernyms())
print("Distància semàntica entre 'restaurant' i 'food': ", np.round(restaurant.wup_similarity(food),2))
print("Distància semàntica entre 'restaurant' i 'menu': ", np.round(restaurant.wup_similarity(menu),2))
print("Distància semàntica entre 'restaurant' i 'waiter': ", np.round(restaurant.wup_similarity(waiter),2))
print("Distància semàntica entre 'restaurant' i 'ambiance': ", np.round(restaurant.wup_similarity(ambiance),2))
print("Distància semàntica entre 'restaurant' i 'reservation': ", np.round(restaurant.wup_similarity(reservation),2))
#############################################



Hiperònims de restaurant:  [Synset('building.n.01')]
Hiperònims de food:  [Synset('substance.n.07')]
Hiperònims de menu:  [Synset('bill.n.07')]
Hiperònims de waiter:  [Synset('dining-room_attendant.n.01')]
Hiperònims de ambiance:  [Synset('condition.n.01')]
Hiperònims de reservation:  [Synset('administrative_district.n.01')]
Distància semàntica entre 'restaurant' i 'food':  0.31
Distància semàntica entre 'restaurant' i 'menu':  0.12
Distància semàntica entre 'restaurant' i 'waiter':  0.42
Distància semàntica entre 'restaurant' i 'ambiance':  0.14
Distància semàntica entre 'restaurant' i 'reservation':  0.38


<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Exercici:</strong> Llista els termes monoparaula del vocabulari de word2vec que no són a Wordnet. Els termes han de ser noms o adjectius. Creieu que aquests termes són importants per poder fer una bona anàlisi de sentiments? (1 punt)

In [72]:
#############################################
# Get the filtered vocabulary that is not in Wordnet
filtered_vocab = [word for word in vocabulary if len(word.split('_')) == 1 
                  and not wn.synsets(word)]

# Get the PoS tags for the filtered vocabulary
pos_tags = pos_tag(filtered_vocab)

# Filter out the words that are not nouns or adjectives
filtered_vocab = [word for (word, pos) in pos_tags if pos.startswith('NN') or pos.startswith('JJ')]

print(filtered_vocab)
#############################################

['everything', 'something', 'scottsdale', 'anything', 'everyone', 'etc', 'pho', 'yum', 'others', 'tempe', 'anyone', 'bruschetta', 'delish', 'lol', 'hey', 'joe', 'crunchy', 'oz', 'st', 'cannot', 'gelato', 'thats', 'meh', 'waitstaff', 'ya', 'yuck', 'bf', 'groupon', 'gf', 'yelpers', 'com', 'torta', 'edamame', 'dont', 'postino', 'ish', 'delux', 'mojito', 'guac', 'boba', 'btw', 'fyi', 'asu', 'fav', 'feta', 'calzone', 'phx', 'didnt', 'carnitas', 'nope', 'coworker', 'rd', 'def', 'cibo', 'cuz', 'um', 'wtf', 'gonna', 'chris', 'eh', 'starbucks', 'mmm', 'horchata', 'ahwatukee', 'everybody', 'fajitas', 'ahi', 'pollo', 'ribeye', 'gotta', 'cheesesteak', 'yasu', 'uh', 'grimaldi', 'vig', 'lil', 'glendale', 'im', 'pupusas', 'ramen', 'yc', 'byob', 'haha', 'roy', 'jeff', 'ive', 'nigiri', 'everytime', 'yep', 'towards', 'ugh', 'chompie', 'chimichanga', 'legit', 'coworkers', 'staffperson', 'burrata', 'denny', 'sandwhich', 'hmm', 'creme', 'sooo', 'rnr', 'gallo', 'ipa', 'alfredo', 'huh', 'masala', 'zinburger'

No sembla haver-hi un tema o categoria específica, però hi ha moltes paraules relacionades amb menjar i restaurants, així com altres paraules i frases col·loquials que es poden utilitzar en converses informals.

Algunes paraules, com "delish" i "yuck", podrien ser útils per a l'anàlisi de sentiments, ja que indiquen una resposta emocional a alguna cosa (en aquest cas, menjar). Altres paraules, com "um" i "ah", poden ser menys útils per a l'anàlisi de sentiments, ja que no indiquen una emoció clara.

En general, la llista conté moltes paraules col·loquials i informals que podrien ser útils en converses informals, però podrien ser menys útils per a l'anàlisi de sentiments en un context més formal.

Hi ha algun pronom com everything, something, anything que no són al wordnet, ja que el WordNet està limitat pel seu àmbit i la seva concentració en la semàntica lèxica, el que significa que es centra principalment en les relacions entre les paraules i els seus significats. Hi ha molts aspectes del llenguatge i el significat que es troben fora de l'àmbit del WordN

## 2.2 LDA (2 punts)

Recordeu que al notebook del mòdul 1 hem vist l'aplicació del mètode LDA per extreure temes de documents.

<i>Primer pas</i>: Convertir les opinions transformades (opinion_sentences_transformed) en llistes de noms i col·locacions. Això és necessari ja que els noms i les col·locacions expressen els temes de les opinions (e.g: [['wife have breakfast sitting_outside'], [''waitress', 'served', 'ceviche']...] -> [ ['wife', 'breakfast', 'sitting_outside'], ['waitress', 'ceviche']]

In [73]:
#############################################
# Preprocessar les dades (només noms i col·locacions)
processed_sentences = []
for sentence in lemmatized_sentences:

    # Filter only nouns and collocations
    processed_words = [word for word in sentence if '_' in word or pos_tag([word])[0][1].startswith('N')]

    # Add the processed sentence to the list
    processed_sentences.append(processed_words)
#############################################

In [74]:
print(processed_sentences[:10])

[['wife_took', 'birthday', 'breakfast', 'excellent', 'weather_perfect', 'sitting_outside', 'overlook', 'ground', 'absolute', 'pleasure_waitress', 'excellent_food', 'arrived_quickly', 'semi', 'busy_saturday', 'morning', 'looked_like', 'place', 'fill', 'pretty_quickly', 'earlier_get', 'favor_get', 'bloody_mary', 'phenomenal', 'simply_best', 'pretty_sure', 'use_ingredients', 'garden', 'blend', 'order', 'amazing_everything', 'menu_looks', 'excellent', 'white_truffle', 'scrambled_eggs', 'vegetable', 'skillet', 'tasty', 'came_pieces', 'griddle', 'bread_amazing', 'absolutely_made', 'meal_complete', 'toast_ever', 'wait_go'], ['idea_people', 'bad_reviews', 'place', 'goes_show', 'please_everyone', 'grip', 'something', 'fault', 'many_people', 'case', 'friend_arrived', 'pm', 'past_sunday', 'pretty_crowded', 'thought', 'sunday_evening', 'thought_would', 'wait', 'forever_get', 'seat', 'said_seated', 'girl', 'comes_back', 'seat', 'someone_else', 'seat', 'waiter_came', 'drink_orders', 'everyone', 'ple

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Exercici:</strong> Extreu temes a partir de la sentència stream d'opinions lematitzades. Experimenta amb el paràmetre num_topics fins a trobar un conjunt de temes informatius assignant noms als temes trobats. (2 punts)
</div>

In [75]:
#############################################
from gensim import corpora
# Crear el diccionari
dictionary = corpora.Dictionary(processed_sentences)

# Crear el corpus
corpus = [dictionary.doc2bow(text) for text in processed_sentences]

# Entrenar el model LDA
num_topics = 10
lda_model = gensim.models.ldamodel.LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=15)

# Mostrar els temes
for idx, topic in lda_model.print_topics(-1):
    print('Tema: {} \nParaules clau: {}'.format(idx + 1, topic))
#############################################

Tema: 1 
Paraules clau: 0.018*"food" + 0.012*"place" + 0.008*"love" + 0.007*"restaurant" + 0.007*"dish" + 0.006*"order" + 0.006*"try" + 0.005*"chicken" + 0.005*"time" + 0.005*"meal"
Tema: 2 
Paraules clau: 0.012*"place" + 0.009*"food" + 0.008*"order" + 0.005*"try" + 0.005*"eat" + 0.004*"time" + 0.004*"restaurant" + 0.004*"menu" + 0.004*"love" + 0.004*"thing"
Tema: 3 
Paraules clau: 0.006*"pizza" + 0.004*"place" + 0.004*"meat" + 0.003*"taste" + 0.003*"cheese" + 0.003*"eat" + 0.002*"order" + 0.002*"chipotle" + 0.002*"side" + 0.002*"lunch"
Tema: 4 
Paraules clau: 0.002*"server_brought" + 0.002*"roy" + 0.002*"deluxe" + 0.002*"best_burger" + 0.002*"got_right" + 0.002*"ingredients_fresh" + 0.002*"super_yummy" + 0.001*"gilbert" + 0.001*"pupusas" + 0.001*"also_wonderful"
Tema: 5 
Paraules clau: 0.014*"place" + 0.011*"food" + 0.009*"time" + 0.006*"restaurant" + 0.005*"menu" + 0.005*"try" + 0.005*"bar" + 0.005*"table" + 0.005*"love" + 0.004*"order"
Tema: 6 
Paraules clau: 0.011*"sandwich" + 0.00

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong> Exercici opcional:</strong> Si considerem també els verbs, milloraríem els resultats? Justifica la resposta mostrant LDA si considerem els verbs, a més dels noms i col·locacions.
</div>

In [76]:
#############################################
# Preprocess the data (adding verbs)
processed_sentences_with_verbs = []
for sentence in lemmatized_sentences:

    # Filter nouns, collocations, and verbs
    processed_words = [word for word in sentence if '_' in word or pos_tag([word])[0][1].startswith('N') or pos_tag([word])[0][1].startswith('V')]

    # Add the processed sentence to the list
    processed_sentences_with_verbs.append(processed_words)
#############################################



In [77]:
#############################################
# Crear el diccionari
dictionary = corpora.Dictionary(processed_sentences_with_verbs)

# Crear el corpus
corpus = [dictionary.doc2bow(text) for text in processed_sentences_with_verbs]

# Entrenar el model LDA
num_topics = 10
lda_model = gensim.models.ldamodel.LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=15)

print("Temes incloent els verbs")
# Mostrar els temes
for idx, topic in lda_model.print_topics(-1):
    print('Tema: {} \nParaules clau: {}'.format(idx + 1, topic))
#############################################

Temes incloent els verbs
Tema: 1 
Paraules clau: 0.003*"great_food" + 0.003*"http_www" + 0.003*"pork_belly" + 0.003*"com" + 0.002*"yelp_com" + 0.002*"trio" + 0.002*"dive_bar" + 0.002*"great_service" + 0.002*"creamed_corn" + 0.002*"central_phoenix"
Tema: 2 
Paraules clau: 0.011*"food" + 0.007*"go" + 0.007*"place" + 0.006*"chicken" + 0.004*"order" + 0.004*"sushi" + 0.004*"salad" + 0.004*"get" + 0.003*"menu" + 0.003*"flavor"
Tema: 3 
Paraules clau: 0.002*"food" + 0.002*"check" + 0.002*"great_happy" + 0.002*"nice_clean" + 0.002*"shake" + 0.002*"bruchetta" + 0.002*"happy_hour" + 0.002*"lo_lo" + 0.002*"gas" + 0.001*"also_excellent"
Tema: 4 
Paraules clau: 0.003*"sandwich" + 0.003*"spa" + 0.003*"tea" + 0.002*"keg" + 0.002*"coconut" + 0.002*"bowl" + 0.002*"buffet" + 0.002*"cookie" + 0.001*"sprout" + 0.001*"daniel"
Tema: 5 
Paraules clau: 0.007*"food" + 0.005*"get" + 0.004*"go" + 0.004*"place" + 0.004*"make" + 0.004*"u" + 0.004*"restaurant" + 0.003*"take" + 0.003*"say" + 0.003*"know"
Tema: 6 
P

**Respecte a si milloraríem els resultats si considerem també els verbs**, dependria del contexte i de l'objectiu específic de la tasca. 


*   Si l'objectiu es extreure temes generals o que no estiguin expecíficament relacionats amb les opinions, no seria necessari incloure els verbs. I en aquest cas, l'enfocament estaria en trobar els patrons comuns de les paraules que es fan servir en els documents.
*    Però, si l'objectiu es extreure temes específics de les opinions, si seria beneficios incloure els verbs, ja que poden proporcionar informació sobre accions, actituts i opinions que expresen en les opinions. Així, els verbs poden indicar si una opinió es positiva o negativa, si expresa una crítica o recomanació o descriu una experiència personal.

 



# 3. Classificació (2 punts)

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Exercici:</strong> Crea un classificador automàtic d'opinions positives i negatives. (0.75 punts)
</div>

<i>Primer pas</i>: realitzem dues llistes. Una amb els textos i una altra amb les etiquetes de valoració (0 i 1)

In [78]:
#############################################
textos = df_star_texts['text'].to_list()
valors = df_star_texts['sentiment'].to_list()
#############################################


<i>Segon pas</i>: Vectoritzem les opinions amb un vectoritzador tf.idf. Feu servir 'word' com a analyzer (0.25 punts)

In [80]:
#############################################
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizerPhrases = TfidfVectorizer(
    analyzer= 'word',                        
 )

X = vectorizerPhrases.fit_transform(textos) # Si transforman los titulares a sus phrases subir nota
textos_vector= X.toarray()
textos_vector
#############################################



array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

<i>Tercer pas</i>: Preparem el corpus d'entrenament i avaluació (0.25 punts)

In [81]:
#############################################
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test  = train_test_split(
        textos_vector, 
        valors,     
        train_size=0.80,
        random_state=1234)
#############################################

<i>Quart pas</i>: Entrenar el classificador amb Logistic Regression

In [82]:
#############################################
from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression()
log_model = classifier.fit(X=X_train, y=y_train)
#############################################

<i>Cinquè pas<i>: Utilitzar el model entrenat per predir la categoria 1 (positiu) o 0 (negatiu) de les opinions del conjunt de test i mostrar les paraules més informatives per a cada categoria. (0.25 punts)

In [38]:
#############################################
y_pred = log_model.predict(X_test)

print("CLASES 0/1 DEL CORPUS DE TEST: ")
print(y_pred)

def most_info_feature_for_binary_classif(vectorizer, classifier, n=10):
    class_labels = classifier.classes_
    feature_names = vectorizer.get_feature_names_out()
    topn_class1 = sorted(zip(classifier.coef_[0], feature_names))[:n]
    topn_class2 = sorted(zip(classifier.coef_[0], feature_names))[-n:]

    for coef, feat in topn_class1:
        print (class_labels[0], coef, feat)

    for coef, feat in reversed(topn_class2):
        print (class_labels[1], coef, feat)

print("\nPARAULES MES INFORMATIVES EN ELS TEXT 0 i 1: \n-------------------------------------------")
print(most_info_feature_for_binary_classif(vectorizerPhrases, classifier))
#############################################


CLASES 0/1 DEL CORPUS DE TEST: 
['0' '1' '1' ... '1' '1' '1']

PARAULES MES INFORMATIVES EN ELS TEXT 0 i 1: 
-------------------------------------------
0 -4.8797840544823385 not
0 -3.3860146662962944 no
0 -2.5740637198308445 bland
0 -2.4737400544839963 ok
0 -2.215749582711953 bad
0 -2.206050788998776 was
0 -2.185509991874954 nothing
0 -2.1762619472026477 tasted
0 -2.16682935533644 just
0 -2.0747856382987098 horrible
1 5.362532533891114 great
1 3.1927342117183226 delicious
1 3.1711260765110296 love
1 3.047153547014852 good
1 3.010416938043725 best
1 2.7233296113464114 and
1 2.714584891076478 amazing
1 2.6146440880470845 definitely
1 2.5684629433538375 awesome
1 2.44615855707341 excellent
None


<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Exercici:</strong> Mostra sobre quins aspectes es fan valoracions negatives. (0.75 punts)
</div>

<i>Primer pas<i>: Tria dues paraules més informatives de la categoria 0 i pren un conjunt d'opinions en què apareguin aquestes paraules. Preprocessa les opinions traient els caràcters de salt de línia (0.25 punts)

In [39]:
#############################################
# SOLUCIÓ                                  # 
#############################################


<i>Segon pas<i>: Utilitza el diccionari d'opinions (arxiu AFINN-111) per extreure la polaritat de cada opinió com la mitjana dels valors de les opinion words del text. (0.25 punts)

In [40]:
#############################################
# SOLUCIÓ                                  #
#############################################



<i>Tercer pas<i>: Selecciona opinions amb polaritat negativa que exemplifiquin els aspectes pitjor valorats. Comenta quins són aquests aspectes (0.25 punts)

In [41]:
#############################################
# SOLUCIÓ                                  #
#############################################



<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Exercici:</strong> Obté els resultats de les mètriques d'avaluació del classificador basat en regressió logística. Obté també els resultats de les mètriques d'avaluació quan el model del classificador és diferent. Per exemple, un model SVM (0.25 punts)
</div>

In [42]:
#############################################
# SOLUCIÓ MODEL REGRESSIÓ LOGÍSTICA                                  #
#############################################



In [43]:
#############################################
# SOLUCIÓ MODEL SVM                         #
#############################################




<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Exercici:</strong> Compara els dos models en funció d'aquestes mètriques d'avaluació. (0.25 punts)
</div>

In [44]:
#############################################
# SOLUCIÓ                                   #
#############################################

